In [1]:
import keras
from keras import backend as K
from keras.layers.core import Dense, Activation
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from keras.layers import Dense,GlobalAveragePooling2D, Dropout
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
import numpy as np
from IPython.display import Image
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
mobile = keras.applications.mobilenet.MobileNet()
def prepare_image(file):
    img_path = ''
    img = image.load_img(img_path + file, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return keras.applications.mobilenet.preprocess_input(img_array_expanded_dims)

In [3]:
base_model=MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
x=Dropout(0.3,name='Droupout')(x) 
preds=Dense(2,activation='softmax')(x) #final layer with softmax activation


/usr/local/lib/python3.5/dist-packages/keras_applications/mobilenet.py:207: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


In [4]:
model=Model(inputs=base_model.input,outputs=preds)
#specify the inputs
#specify the outputs
#now a model has been created based on our architecture
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, None, None, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (ReLU)            (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, None, None, 32)    128       
__________

In [5]:
#for i,layer in enumerate(model.layers):
    #print(i,layer.name)

In [6]:
for layer in model.layers:
    layer.trainable=False
# or if we want to set the first 20 layers of the network to be non-trainable
for layer in model.layers[:70]:
    layer.trainable=False
for layer in model.layers[70:]:
    layer.trainable=True

In [8]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True) #included in our dependencies

train_generator=train_datagen.flow_from_directory('/home/himani/MobileNet/profile/train',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=10,
                                                 class_mode='categorical',shuffle=True)

Found 1861 images belonging to 2 classes.


In [9]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

step_size_train=train_generator.n//train_generator.batch_size
model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train, epochs=10)

Epoch 1/10
  1/186 [..............................] - ETA: 10:37 - loss: 1.3697 - acc: 0.3000

/home/himani/.local/lib/python3.5/site-packages/PIL/Image.py:953: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


186/186 [==============================] - 113s 607ms/step - loss: 0.1306 - acc: 0.9688
Epoch 2/10
186/186 [==============================] - 118s 635ms/step - loss: 0.0538 - acc: 0.9898
Epoch 3/10
186/186 [==============================] - 106s 570ms/step - loss: 0.0140 - acc: 0.9978
Epoch 4/10
186/186 [==============================] - 105s 565ms/step - loss: 0.1943 - acc: 0.9758
Epoch 5/10
186/186 [==============================] - 107s 573ms/step - loss: 0.0234 - acc: 0.9962
Epoch 6/10
186/186 [==============================] - 114s 615ms/step - loss: 0.2821 - acc: 0.9699
Epoch 7/10
186/186 [==============================] - 112s 603ms/step - loss: 0.1048 - acc: 0.9930
Epoch 8/10
186/186 [==============================] - 111s 595ms/step - loss: 0.1160 - acc: 0.9914
Epoch 9/10
186/186 [==============================] - 104s 557ms/step - loss: 0.0785 - acc: 0.9935
Epoch 10/10
186/186 [==============================] - 104s 559ms/step - loss: 0.1343 - acc: 0.9860


In [11]:
def load_image(img_path, show=False):

    img = image.load_img(img_path, target_size=(150, 150))
    img_tensor = image.img_to_array(img)                    # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    img_tensor /= 255.                                      # imshow expects values in the range [0, 1]

    if show:
        plt.imshow(img_tensor[0])                           
        plt.axis('off')
        plt.show()

    return img_tensor
  
#img_path = '/home/himani/bald_imgs_26dec/Young businessman making a stupid face. Studio shot on black background..jpg'
#img_path = '/home/himani/MobileNet/HUMAN/ZenH_02659_f_25_i_nf_nc_hp_2015_1_e0_nl_o.jpg'
img_path = '/home/himani/MobileNet/top5dl-bn.jpg'
#img_path = '/home/himani/MobileNet/crow.jpeg'
#img_path = '/home/himani/bald_imgs_26dec/0e654e23d1acbb9a0a754cb4d1629db4.jpg'
new_image = load_image(img_path)

pred = model.predict(new_image)

pred

FileNotFoundError: [Errno 2] No such file or directory: '/home/himani/MobileNet/top5dl-bn.jpg'